# Scratch ReAct agent

In [1]:
from dotenv import load_dotenv
import sys
import json

from langchain.prompts import PromptTemplate

# Load the file that contains the API keys - OPENAI_API_KEY
load_dotenv('C:\\Users\\raj\\.jupyter\\.env')

# setting path
sys.path.append('../')

from utils.create_chat_llm import create_gpt_chat_llm, create_cohere_chat_llm, create_anthropic_chat_llm, create_hugging_face_chat_llm
from utils.create_llm import create_cohere_llm, create_gpt_llm

# Try with GPT
llm = create_gpt_chat_llm({"temperature":0.0}) #, "model_kwargs":{"top_p":0.1}})

In [2]:
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

wikipedia_tool = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

from langchain.agents import load_tools

serp_tool = load_tools(["serpapi"])[0]

from langchain_community.tools.tavily_search import TavilySearchResults

tavily_tool = TavilySearchResults()

# Create tools representation for the prompt

# {
#     "name" : 'serpapi',
#     "description": "use for searching the internet for answers",
#     "arguments": [
#         {"input" : "input"}
#     ],
#     "response": "search results"
# },

tools = [{
    "name" : 'wikipedia',
    "description": wikipedia_tool.description,
    "arguments": [
        {"input" : "input"}
    ],
    "response": "search results"
}, {
    "name" : 'tavily',
    "description": tavily_tool.description,
    "arguments": [
        {"input" : "input"}
    ],
    "response": "search results"
}
]

tools = json.dumps(tools, indent=4)

tools_map = {
    'wikipedia': wikipedia_tool.invoke,
    'serpapi' : serp_tool.invoke,
    'tavily' : tavily_tool.invoke
}



In [3]:
print(tools)

[
    {
        "name": "wikipedia",
        "description": "A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.",
        "arguments": [
            {
                "input": "input"
            }
        ],
        "response": "search results"
    },
    {
        "name": "tavily",
        "description": "A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.",
        "arguments": [
            {
                "input": "input"
            }
        ],
        "response": "search results"
    }
]


## 1. ReAct prompt

In [4]:
prompt_template = """
You are a helpful assistant capable of answering questions on various topics. 
Do not use your internal knowledYou MUST validate the answer using tools to access external tools.

Instructions:

Take an iterative approach. In each iteration:
1. Thought : Think step by step. Use alternatives as needed.
2. Action : Execute appropriate tools. Try alternative tool if an appropriate response is not received.
3. Observations : Make observations based on the responses from the tools and decide the next step

Repeat this Thought/Action/Observation process N times till either you get the answer or are unable to get the answer using the available tools.

Use only the following available tools to find information.

Tools Available:


{tools}
Guidelines for Responses:

Format 1: If the question cannot be answered with the available tools, use this format:
{{
    "answer": "No appropriate tool available",
    "scratchpad": {{
        "thought": "your inner thoughts",
        "action": "tool name",
        "observations": "your observations from the tool responses"
    }}
}}

Format 2: If you need to run tools to obtain the information, use this format:
{{
    "actions": [
        {{
            "action": tool name,
            "arguments": dictionary of argument values
        }},
        "scratchpad": {{
            "thought": "your inner thoughts",
            "action": "tool name",
            "observations": "your observations from the tool responses"
        }}
    ]
}}

Format 3: If you can answer the question using the responses from the tools, use this format:
{{
    "answer": "your response to the question",
    "scratchpad": {{
        "thought": "your inner thoughts",
        "action": "tool name",
        "observations": "your observations from the tool responses"
    }}
}}

Avoid any preamble; respond directly using one of the specified JSON formats.
Question:

{question}
Tool Responses:

{tool_responses}
Your Response:
"""

prompt_template_1 = """
You are a helpful assistant capable of answering questions on various topics. You must validate your answers using external tools.

Instructions:

Take the following iterative approach:

Thought: Think step-by-step and consider alternatives as needed.
Action: Execute or more tools. Try alternative tools if the tool you selected did not provide a response that you could use or if the tool response is insufficent for response. If needed adjust the argument values for tools.
Observations: Make observations based on the tool responses and decide the next step.
......(Repeat this Thought/Action/Observation cycle until you either have the answer or reach a conclusion that it is not possible using the available tools)

Tools:

{tools}

Guidelines for Responses:

Format 1: If the question cannot be even after using the available tools, use this format:

{{
    "answer": "No appropriate tool available",
    "scratchpad": {{
        "thought": "your inner thoughts",
        "action": "tool name",
        "observations": "your observations based on the tool responses"
    }}
}}
Format 2: If you need to run tools to obtain the information, use this format:

{{
    "actions": [
        {{
            "action": "tool name",
            "arguments": {{"key": "value"}}
        }}
    ],
    "scratchpad": {{
        "thought": "your inner thoughts",
        "action": "tool name",
        "observations": "your observations from the tool responses"
    }}
}}

Format 3: when you finally have the answer, use this format:

{{
    "answer": "your response to the question",
    "scratchpad": {{
        "thought": "your inner thoughts",
        "action": "tool name",
        "observations": "your observations from the tool responses"
    }}
}}

Avoid any preamble; respond directly using one of the specified JSON formats.

Question:
{question}

Tool responses:
{tool_responses}

Your Response:
"""

prompt = PromptTemplate(
    template = prompt_template,
    input_variables = ["tools", "question"]
)

In [5]:
# Utility function for answering the question
def  invoke_agent(input):
    
    # Setup the prompt. Since no tool has been invoked set action_response as blank
    # query = prompt.format(tools=tool_description, question=question, tool_responses="")

    # STEP-1 Invoke LLM for a plan i.e., tools to execute
    # ===================================================
    # Invoke LLM to get the tools to be run
    # The response consist of tools that LLM requires to be executed
    response = llm.invoke(input)

    print("LLM Response: ", response.content)
    
    # Convert response to JSON object. The response is of type AIMessage
    response_json = json.loads(response.content)

    
    action_responses=[]
    if "answer" in response_json:
        # If the answer is already there
        return response.content, {"answer" : response_json["answer"]}
    elif "actions" in response_json:
        # If the LLM has suggested tools to be executed, execute the tools
        action_responses = { "action_responses": invoke_tools(response_json)}

    return response.content, action_responses


In [6]:
def   invoke_tools(response):
    action_responses = []
    if len(response["actions"]) == 0:
        print('question cannot be answered as there is no tool to use !!!')
        exit
    else:
        for action in response["actions"]:

            # Get the function pointer from the map
            action_function = tools_map[action["action"]]
            
            # Invoke the tool/function with the arguments as suggested by the LLM
            action_invoke_result = action_function(**action["arguments"])
            action["action_response"] = action_invoke_result

            # Add the response to the action attribute
            action_responses.append(action)

    # Return the response      
    return action_responses

In [11]:
question = "who is older Issac Newton or Albert Einstien?"

# question = "Were Daniel Mann and Todd Solondz both directors?"

# question = "Author David Chanoff has collaborated with a U.S. Navy admiral who served as the ambassador to the United Kingdom under which President?"

# Q: Where is the basketball team that Mike DiNunno plays for based ?
# A: Ellesmere Port
# question = "Where is the basketball team that Mike DiNunno plays for based ?"

# question = "Peter Marc Jacobson is best known as the co-creator of the popular sitcom 'The Nanny', which he created and wrote with his then wife an actress born in which year ?"

# Q: what type of media does Ratatouille and PlayStation 3 have in common?
# A: video game
# question = "what type of media does Ratatouille and PlayStation 3 have in common?"

# Q: Risingson is the first single from what album by Massive Attack, that was the first to be produced by Neil Davidge, along with the group?
# A: Mezzanine
question = "Risingson is the first single from what album by Massive Attack, that was the first to be produced by Neil Davidge, along with the group?"

# Q: What role on "Switched at Birth" was played by the actor who is being replaced by Daniel Hall on "The Young and the Restless"?
# A: Tyler "Ty" Mendoza
# question = 'What role on "Switched at Birth" was played by the actor who is being replaced by Daniel Hall on "The Young and the Restless"?'

# Q: Are Pago Pago International Airport and Hoonah Airport both on American territory?
# A: yes
# question = "Are Pago Pago International Airport and Hoonah Airport both on American territory?"

# Q: Where did the character Xavin changed into a black female just for the sake first appeared
# A: "Runaways".
# question = "Where did the character Xavin changed into a black female just for the sake first appeared"

input = prompt.format(tools=tools, question=question, tool_responses='')

In [12]:
continue_iteration = True

iteration_number = 0
while continue_iteration:

    iteration_number = iteration_number + 1
    print("ITERATION# : ", iteration_number )
    
    
    llm_response, action_response = invoke_agent(input)
    
    # print("action_response====",action_response)

    # response_json = json.loads(action_response)
    print("============================================")
    if "answer" in action_response:
        answer = action_response['answer']
        continue_iteration = False
    else:
        input = input + "\n"+ json.dumps(llm_response) + "\nTool responses: \n" + json.dumps(action_response) + "\n Your response: "

ITERATION# :  1
LLM Response:  {
    "actions": [
        {
            "action": "wikipedia",
            "arguments": {
                "input": "Risingson Massive Attack album"
            }
        }
    ],
    "scratchpad": {
        "thought": "Let's search for information about the album that contains the song 'Risingson' by Massive Attack and was produced by Neil Davidge.",
        "action": "wikipedia",
        "observations": "Awaiting response from Wikipedia search."
    }
}
ITERATION# :  2
LLM Response:  {
    "answer": "Mezzanine",
    "scratchpad": {
        "thought": "The album that contains the song 'Risingson' by Massive Attack and was produced by Neil Davidge is Mezzanine.",
        "action": "wikipedia",
        "observations": "The Wikipedia search confirmed that 'Risingson' is from the album Mezzanine."
    }
}


In [9]:
type(tools[0])

str

In [10]:
print(response)

NameError: name 'response' is not defined

In [ ]:
print(response['answer'])

In [ ]:
response_json

In [ ]:
from langchain_community.utilities import SerpAPIWrapper

search = SerpAPIWrapper()

search.run("Obama's first name?")

In [ ]:
from langchain.agents import load_tools

tools = load_tools(["serpapi"])

In [ ]:
tools[0].invoke(input="what is the date today")

In [ ]:
serp_tool.description

In [ ]:
tool_responses

In [22]:
from langchain_community.tools.tavily_search import TavilySearchResults

tool = TavilySearchResults(include_raw_content=False, include_answer=True, max_results=3)

tool.description

'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.'

In [23]:
tool.invoke("Mike DiNunno basketball team")

[{'url': 'https://en.wikipedia.org/wiki/Mike_DiNunno',
  'content': 'Mike DiNunno. Michele Christopher Di Nunno (born January 29, 1990) is an American-Italian professional basketball player, playing the point guard position. Born in Maywood, Illinois, he played high school basketball at Lake Park and Von Steuben.'},
 {'url': 'https://ekusports.com/sports/mens-basketball/roster/mike-dinunno/4250',
  'content': 'Mike DiNunno (3) Guard - Neubauer on DiNunno: "Mike is a proven double-figure scorer at the Division I level. ... 2012-13 Men\'s Basketball Roster. Choose a Player: Go. Terrence Humphrey ... Played in all 30 games with 21 starts … finished second on the team in both assists (91) and three-pointers made (42) … ranked fourth on the squad in ...'},
 {'url': 'https://www.espn.com/mens-college-basketball/player/_/id/41244/mike-dinunno',
  'content': 'View the profile of Eastern Kentucky Colonels Guard Mike DiNunno on ESPN. Get the latest news, live stats and game highlights.'}]

In [ ]:
print(input)

In [ ]:
llm_response

In [ ]:
import time
round(time.time() * 1000)